# Working with multiple phases

This notebook will explain how to load, access and fit multiple phases

#### Import Python packages

In [ ]:
# esyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# esyScience, diffraction
from easyDiffractionLib import Phases
from easyDiffractionLib.sample import Sample as Job
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters as CWParams

# Vizualization
import py3Dmol
import matplotlib.pyplot as plt

## --- Sample ---

#### Show a CIF file content

In [ ]:
cif_fname = 'multiphase.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

 This cif file contains two phases of Si3N4: `alpha` and `beta`.

#### Load structure from a CIF file

In [ ]:
phases = Phases.from_cif_file(cif_fname)
phase_alpha = phases[0]
phase_beta = phases[1]

print(phases)
print(phase_alpha)
print(phase_beta)

#### Visualise the first phase

In [ ]:
structure = py3Dmol.view()
structure.addModel(phase_alpha.to_cif_str(), 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

#### Visualise the second phase

In [ ]:
structure_2 = py3Dmol.view()
structure_2.addModel(phase_beta.to_cif_str(), 'cif')
structure_2.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure_2.addUnitCell()
structure_2.replicateUnitCell(2,2,1)
structure_2.zoomTo()

## --- Experiment ---

#### Load the measured data

In [ ]:
meas_fname = '3T2@LLB.xye'
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

#### Visualize the measured data

In [ ]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.legend()

## --- Analysis ---

#### Create job with default parameters for the 1D powder neutron diffraction experiment with constant wavelength 

In [ ]:
calculator = Calculator(interface_name='CrysPy')

In [ ]:
print(f"Current calculator engine: {calculator.current_interface_name}")

In [ ]:
job = Job(phases=phases, parameters=CWParams.default(), calculator=calculator)

#### Generate the calculated data

**Note**: *Calculated data corresponds to the sum of all phases*

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

#### Visualize both the measured and calculated data

In [ ]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### We can also view separate phases contributions

In [ ]:
y_phase_1 = calculator.get_calculated_y_for_phase(0)
y_phase_2 = calculator.get_calculated_y_for_phase(1)

In [ ]:
%matplotlib widget
plt.plot(meas_x, y_phase_1, label='Si3N4 alpha')
plt.plot(meas_x, y_phase_2, label='Si3N4 beta')
plt.legend()

#### Set scale manually, for each phase separately

In [ ]:
phases[0].scale = 92.
phases[1].scale = 28.6

#### Set wavelength manually

In [ ]:
job.parameters.wavelength = 1.2251

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

calc_y_cryspy = calculator.get_total_y_for_phases()[1]
calc_y_cryspy = calculator.get_calculated_y_for_phase(1)
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Set background points manually

In [ ]:
bkg = PointBackground(linked_experiment='PbSO4')

bkg.append(BackgroundPoint.from_pars(meas_x[0], 200))
bkg.append(BackgroundPoint.from_pars(meas_x[-1], 250))

job.set_background(bkg)

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Define parameters to optimize

In [ ]:
job.phases[0].scale.fixed = False
job.phases[1].scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.backgrounds[0][0].y.fixed = False
job.backgrounds[0][1].y.fixed = False

In [ ]:
print(job.phases[0].scale)
print(job.phases[1].scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.backgrounds[0][0])
print(job.backgrounds[0][1])

#### Initalize the fitting engine and perform the fit

In [ ]:
fitter = Fitter(job, calculator.fit_func)

In [ ]:
print(f"Available minimizers: {fitter.available_engines}")
print(f"Current minimizer: {fitter.current_engine.name}")
print(f"Available methods of current minimizers: {fitter.available_methods()}")

In [ ]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, 
                    method='least_squares', minimizer_kwargs={'diff_step': 1e-5})

In [ ]:
print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, meas_y-calc_y_cryspy, label='Imeas - Icalc (CrysPy)')
plt.legend()